In [ ]:
#A. Read the Data

In [2]:
import pandas as pd

df = pd.read_csv(r'C:\Users\Viyok\Downloads\Credit.csv')
df.head()

,Income,Limit,Rating,Cards,Age,Education,Married,Balance
0,14.891,3606,283,2,34,11,1,333
1,106.025,6645,483,3,82,15,1,903
2,104.593,7075,514,4,71,11,0,580
3,148.924,9504,681,3,36,11,0,964
4,55.882,4897,357,2,68,16,1,331


In [ ]:
#B. Check the datset's strucuture

In [6]:
# Check the dataset's structure
df.info()

# Display summary statisitics for numerical columns
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Income     400 non-null    float64
 1   Limit      400 non-null    int64  
 2   Rating     400 non-null    int64  
 3   Cards      400 non-null    int64  
 4   Age        400 non-null    int64  
 5   Education  400 non-null    int64  
 6   Married    400 non-null    int64  
 7   Balance    400 non-null    int64  
dtypes: float64(1), int64(7)
memory usage: 25.1 KB


,Income,Limit,Rating,Cards,Age,Education,Married,Balance
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.00000,400.000000
mean,45.218885,4735.600000,354.940000,2.957500,55.667500,13.450000,0.61250,520.015000
std,35.244273,2308.198848,154.724143,1.371275,17.249807,3.125207,0.48779,459.758877
min,10.354000,855.000000,93.000000,1.000000,23.000000,5.000000,0.00000,0.000000
25%,21.007250,3088.000000,247.250000,2.000000,41.750000,11.000000,0.00000,68.750000
50%,33.115500,4622.500000,344.000000,3.000000,56.000000,14.000000,1.00000,459.500000
75%,57.470750,5872.750000,437.250000,4.000000,70.000000,16.000000,1.00000,863.000000
max,186.634000,13913.000000,982.000000,9.000000,98.000000,20.000000,1.00000,1999.000000


In [ ]:
#C.Generate the feature matrix and target vector (target is “balance” in this dataset). 

In [10]:
# Selecting specific columns for the feature matrix
feature_cols = ['Income', 'Limit', 'Rating', 'Cards', 'Age', 'Education', 'Married']
X = df[feature_cols]  # Feature matrix
y = df['Balance']  # Target vector

# Display the first 5 rows of the feature matrix
print("Feature Matrix (first 5 rows):")
print(X.head())

Feature Matrix (first 5 rows):
    Income  Limit  Rating  Cards  Age  Education  Married
0   14.891   3606     283      2   34         11        1
1  106.025   6645     483      3   82         15        1
2  104.593   7075     514      4   71         11        0
3  148.924   9504     681      3   36         11        0
4   55.882   4897     357      2   68         16        1


In [14]:
#Target vector
y = df[['Balance']]
y.head()

,Balance
0,333
1,903
2,580
3,964
4,331


In [ ]:
#D.Split the dataset into testing and training sets with the following parameters: test_size=0.24, random_state=9.

In [12]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.24, random_state=9)

In [18]:
#E.Now, normalize (scale) the features. To normalize the data, you can simply use StandardScaler  from sklearn. (note: don’t normalize the target!). 
#Remember that we can only use the statistics of X_train for normalization, and then apply it on both X_train and X_test:

In [20]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler and fit it on the training data only
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)


In [22]:
#F.Use Linear Regression to train a linear model on the training set. Check the coefficients of the linear regression model. 
# Which feature is the most important? Which feature is the least important? 

In [24]:
from sklearn.linear_model import LinearRegression

# Initialize and train the linear regression model
linear_regression = LinearRegression()
linear_regression.fit(X_train_normalized, y_train)

# Retrieve and display feature coefficients
coefficients = pd.Series(linear_regression.coef_, index=X.columns)
coefficients_sorted = coefficients.sort_values(ascending=False)

print("Feature Importance (sorted by coefficient):")
print(coefficients_sorted)


Feature Importance (sorted by coefficient):
Limit        373.757958
Rating       261.945404
Cards         24.838103
Education     11.551102
Married       -9.901947
Age          -17.508705
Income      -280.210428
dtype: float64


In [ ]:
#G.Predict Balance on the Testing Set and Calculate RMSE

In [26]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Predict the balance for the testing set
y_pred = linear_regression.predict(X_test_normalized)

# Calculate the Root Mean Square Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE on Testing Set: {rmse:.2f}")

RMSE on Testing Set: 143.25


In [ ]:
#H.Use 10-Fold Cross-Validation to Evaluate Performance with Normalization

In [28]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

# Create a pipeline that combines normalization and linear regression
pipeline = make_pipeline(StandardScaler(), LinearRegression())

# Perform 10-fold cross-validation with RMSE as the scoring metric
mse_scores = cross_val_score(pipeline, X, y, cv=10, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-mse_scores)  # Convert negative MSE to positive RMSE

# Calculate and print the average RMSE across all folds
print("Cross-Validation RMSE for each fold:", rmse_scores)
print(f"Average RMSE from 10-Fold Cross-Validation: {rmse_scores.mean():.2f}")

Cross-Validation RMSE for each fold: [153.7754992  178.89394627 188.31527914 193.20353826 119.75525901
 183.38039983 177.85385343 111.76315739 144.04725656 152.331702  ]
Average RMSE from 10-Fold Cross-Validation: 160.33
